In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import uuid

In [4]:
# !pip install surt
# !pip install validators

In [5]:
# Two handy libraries for checking if a URL is valid or not
import surt
import validators

## SEC-Registered

In [6]:
df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_03_20_2023.csv')

C:\Users\14097\AppData\Local\Temp\ipykernel_15868\926076033.py:1: DtypeWarning: Columns (49,217,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_03_20_2023.csv')


In [7]:
# Remove long prefix at the beginning of many column names
df_sec.columns = df_sec.columns.str.removeprefix('FormInfo.Part1A.Item')

In [8]:
df_sec['uuid'] = df_sec.apply(lambda _: uuid.uuid4(), axis=1)

df_sec['uuid']

0        82c31e31-f06d-4957-8df9-75fbc8fc9b60
1        87a09822-ae37-4776-a471-fb2a1487e008
2        d9fc43de-962d-4a74-a8e2-8c56890d561c
3        8c7a256b-eeeb-4237-8331-4eaf06fe94c9
4        8f187cf8-fec7-463d-899b-9ae8504f2ba4
                         ...                 
21095    f5a0186b-8439-45a7-b3d7-18cc73fcf479
21096    e451ccd0-1a85-43ab-a186-4df99b7bf93d
21097    6c71871f-26b9-44f9-bb77-9d5dbf459dd5
21098    f09d5938-88ac-4588-9576-646e42c326b2
21099    c302b327-37ab-4115-8384-508cf7d4fb04
Name: uuid, Length: 21100, dtype: object

In [9]:
sorted(df_sec.columns.to_list())

['1.@Q1F5',
 '1.@Q1I',
 '1.@Q1M',
 '1.@Q1N',
 '1.@Q1O',
 '1.@Q1ODesc',
 '1.@Q1P',
 '1.WebAddrs',
 '1.WebAddrs.WebAddr',
 '10A.@Q10A',
 '11.@Q11',
 '11A.@Q11A1',
 '11A.@Q11A2',
 '11B.@Q11B1',
 '11B.@Q11B2',
 '11C.@Q11C1',
 '11C.@Q11C2',
 '11C.@Q11C3',
 '11C.@Q11C4',
 '11C.@Q11C5',
 '11D.@Q11D1',
 '11D.@Q11D2',
 '11D.@Q11D3',
 '11D.@Q11D4',
 '11D.@Q11D5',
 '11E.@Q11E1',
 '11E.@Q11E2',
 '11E.@Q11E3',
 '11E.@Q11E4',
 '11F.@Q11F',
 '11G.@Q11G',
 '11H.@Q11H1A',
 '11H.@Q11H1B',
 '11H.@Q11H1C',
 '11H.@Q11H2',
 '2A.@Q2A1',
 '2A.@Q2A10',
 '2A.@Q2A11',
 '2A.@Q2A12',
 '2A.@Q2A13',
 '2A.@Q2A2',
 '2A.@Q2A4',
 '2A.@Q2A5',
 '2A.@Q2A6',
 '2A.@Q2A7',
 '2A.@Q2A8',
 '2A.@Q2A9',
 '2B.@Q2B1',
 '2B.@Q2B2',
 '2B.@Q2B3',
 '3A.@OrgFormNm',
 '3A.@OrgFormOthNm',
 '3B.@Q3B',
 '3C.@CntryNm',
 '3C.@StateCD',
 '5A.@TtlEmp',
 '5B.@Q5B1',
 '5B.@Q5B2',
 '5B.@Q5B3',
 '5B.@Q5B4',
 '5B.@Q5B5',
 '5B.@Q5B6',
 '5C.@Q5C1',
 '5C.@Q5C1Oth',
 '5C.@Q5C2',
 '5D.@Q5D1A',
 '5D.@Q5D1B',
 '5D.@Q5D1C',
 '5D.@Q5D1D',
 '5D.@Q5D1E',
 '5D.@

Number of Advisers -- everything

In [10]:
df_sec.shape

(21100, 279)

In [11]:
df_sec['Info.@SECNb'].unique().size

21100

In [12]:
df_secno = df_sec['Info.@SECNb']

In [13]:
df_secno

0         801-73527
1        801-118322
2        801-106933
3        801-108550
4         801-18270
            ...    
21095    801-127344
21096    802-127293
21097    801-127370
21098    801-127237
21099    801-127348
Name: Info.@SECNb, Length: 21100, dtype: object

In [14]:
df_secnb = df_sec['Info.@FirmCrdNb']

In [15]:
df_secnb.shape[0]

21100

In [16]:
df_sec['large_or_mid_sized'] = np.where((df_sec['2A.@Q2A1'] == 'Y') | (df_sec['2A.@Q2A2'] == 'Y'),
                                        True,
                                        False)

In [17]:
df_sec['robo'] = np.where(df_sec['2A.@Q2A11'] == 'Y',
                          True, False)

In [18]:
(df_sec.loc[(df_sec['2A.@Q2A1'] == 'Y') | (df_sec['2A.@Q2A2'] == 'Y'),:].index == df_sec[df_sec['large_or_mid_sized']].index).all()

True

Number of Advisers -- just the "regular" firms

In [19]:
df_sec_reg = df_sec[df_sec['large_or_mid_sized']].copy()
df_sec_reg.shape[0]

13797

Number of Robo Advisers

In [20]:
df_sec_robo = df_sec[df_sec['robo']].copy()
df_sec_robo.shape[0]

270

## State-Registered Advisers 

In [21]:
df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_03_20_2023.csv')

C:\Users\14097\AppData\Local\Temp\ipykernel_15868\2016899816.py:1: DtypeWarning: Columns (35,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,270,277,306,309,310,313) have mixed types. Specify dtype option on import or set low_memory=False.
  df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_03_20_2023.csv')


In [22]:
sorted(df_state.columns.to_list())

['ERA.Rgltrs',
 'ERA.Rgltrs.Rgltr',
 'ERA.Rgltrs.Rgltr.@Cd',
 'ERA.Rgltrs.Rgltr.@Dt',
 'ERA.Rgltrs.Rgltr.@St',
 'Filing.@Dt',
 'Filing.@FormVrsn',
 'FormInfo.Part1A.Item1.@Q1F5',
 'FormInfo.Part1A.Item1.@Q1I',
 'FormInfo.Part1A.Item1.@Q1M',
 'FormInfo.Part1A.Item1.@Q1N',
 'FormInfo.Part1A.Item1.@Q1O',
 'FormInfo.Part1A.Item1.@Q1ODESC',
 'FormInfo.Part1A.Item1.@Q1P',
 'FormInfo.Part1A.Item1.WebAddrs',
 'FormInfo.Part1A.Item1.WebAddrs.WebAddr',
 'FormInfo.Part1A.Item10A.@Q10A',
 'FormInfo.Part1A.Item11.@Q11',
 'FormInfo.Part1A.Item11A.@Q11A1',
 'FormInfo.Part1A.Item11A.@Q11A2',
 'FormInfo.Part1A.Item11B.@Q11B1',
 'FormInfo.Part1A.Item11B.@Q11B2',
 'FormInfo.Part1A.Item11C.@Q11C1',
 'FormInfo.Part1A.Item11C.@Q11C2',
 'FormInfo.Part1A.Item11C.@Q11C3',
 'FormInfo.Part1A.Item11C.@Q11C4',
 'FormInfo.Part1A.Item11C.@Q11C5',
 'FormInfo.Part1A.Item11D.@Q11D1',
 'FormInfo.Part1A.Item11D.@Q11D2',
 'FormInfo.Part1A.Item11D.@Q11D3',
 'FormInfo.Part1A.Item11D.@Q11D4',
 'FormInfo.Part1A.Item11D.@Q11D5

In [23]:
df_state['uuid'] = df_state.apply(lambda _: uuid.uuid4(), axis=1)

In [24]:
df_state['uuid']

0        d9c422f1-f5ce-4d8b-9447-abde4e9575c3
1        e6fdbd2a-ecc9-4c24-a425-52d7e22b1991
2        5d52ddea-9f95-48fe-af59-9a66cb5242e4
3        3b629316-c821-4548-b00a-07e6fa8c7f4a
4        db942b51-7473-44e7-966d-25964f7f3fe1
                         ...                 
21261    76d9efc0-e7a3-416d-83c4-dff1bbb770e4
21262    db71a4ad-7b86-4760-8476-e688fcd0e46b
21263    b0b31cd6-cf33-4a16-a532-3aac5d54b843
21264    3e18d95c-dca3-4e7f-a1d2-cc6db7cd2ca2
21265    27c072ad-a51f-4bc7-b249-7afca65a7941
Name: uuid, Length: 21266, dtype: object

Some Advisers are "Exempt Reporting Advisers", they are typically VCs 
https://learn.angellist.com/articles/exempt-reporting-advisor

In [25]:
df_state_exempt = df_state[(df_state['ERA.Rgltrs'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Cd'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Dt'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@St'].notnull() )].copy()

In [26]:
df_state_reg = df_state[(df_state['ERA.Rgltrs'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Cd'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Dt'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@St'].isnull() )].copy()

In [27]:
df_state_reg.shape[0] + df_state_exempt.shape[0] == df_state.shape[0]

True

In [28]:
df_state.shape[0]

21266

In [29]:
df_state.shape

(21266, 315)

In [30]:
df_state['Info.@SECNb'].unique().size

3862

In [31]:
df_state['Info.@FirmCrdNb'].unique().size

21266

In [32]:
df_statecrb = df_state['Info.@FirmCrdNb']

In [33]:
df_statecrb

0        146099
1        289221
2         23940
3        155220
4        171863
          ...  
21261    324629
21262    324630
21263    324661
21264    324664
21265    324649
Name: Info.@FirmCrdNb, Length: 21266, dtype: int64

# Combine

In [34]:
df_all = pd.merge(df_sec, df_state, on=['Info.@FirmCrdNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [35]:
df_all.shape

(1787, 593)

In [36]:
df_all['Info.@FirmCrdNb'].unique().size

1787

In [37]:
df_all['Info.@SECNb_x'].unique().size

1787

In [38]:
df_all.sample(3)

,Info.@SECRgnCD,Info.@FirmCrdNb,Info.@SECNb_x,Info.@BusNm,Info.@LegalNm,Info.@UmbrRgstn_x,MainAddr.@Strt1_x,MainAddr.@Strt2_x,MainAddr.@City_x,MainAddr.@State_x,...,FormInfo.Part1B.ItemJ.@Q1BJ2BNone,FormInfo.Part1A.Item9E.@Q9E,ERA.Rgltrs.Rgltr,FormInfo.Part1B.ItemH.@Q1B2HNScrtsNvsmtAm,FormInfo.Part1A.Item1.@Q1ODESC,FormInfo.Part1A.Item5D.@Q5D1MOth,FormInfo.Part1A.Item5H.@Q5HMT500,FormInfo.Part1A.Item5C.@Q5C1Oth,FormInfo.Part1A.Item5D.@Q5D2MOth,uuid_y
387,NYRO,163343,802-75144,"NEW YORK CITY INVESTMENT FUND MANAGER, INC.","NEW YORK CITY INVESTMENT FUND MANAGER, INC.",NaN,ONE BATTERY PARK PLAZA,NaN,NEW YORK,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8f61d32e-94a0-43f4-baad-ad4d39f10ba9
476,LARO,289457,802-112715,BONFIRE VENTURES,"BONFIRE VENTURES MANAGEMENT, LLC",NaN,725 ARIZONA AVE.,SUITE 400,SANTA MONICA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98a77ffb-78af-4072-88fc-f7ddf88a6c22
749,SFRO,162705,802-75503,DAG VENTURES,"DAG VENTURES, LLC",NaN,251 LYTTON AVE.,SUITE 200,PALO ALTO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a48a9fda-c27d-455f-a36c-186923906ebe


# Get All Web Addresses

In [39]:
def get_all_urls(df, valid_only=True):
    df_urls = df.loc[df['1.WebAddrs.WebAddr'].notnull(),
                    ['Info.@FirmCrdNb',
                     '1.WebAddrs.WebAddr']
                    ]
    df_urls.rename(columns={'1.WebAddrs.WebAddr':'url'}, inplace=True)
    df_urls['url'] = df_urls['url'].apply(lambda x: eval(x.lower()) if ('[' in x and ']' in x) else x.lower())
    df_urls = df_urls.explode('url').reset_index(drop=True)
    df_urls['url_valid'] = df_urls['url'].apply(validators.url)
    if valid_only:
        df_urls = df_urls.loc[df_urls['url_valid'] == True,:]
    return df_urls

In [40]:
df_sec_reg_urls = get_all_urls(df_sec_reg, valid_only=False)